In [15]:
# doing the classification on the urban 

In [16]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import matplotlib.pyplot as plt
import csv
import os
import numpy as np
import collections

import scipy.io.wavfile
import collections
import wavio

import sys, os
sys.path.append(os.path.expanduser('~/projects/engaged_hackathon/'))
from engaged.features import frequency
import IPython

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
base_path = '/home/michael/projects/engaged_hackathon_data/urban_8k/'
meta_path = base_path + 'UrbanSound8K/metadata/UrbanSound8K.csv'
wav_path = base_path + 'UrbanSound8K/audio/'

spectrogram_parameters = {
    'nfft': 512,
    'window_width': 0.03,
    'overlap': 0.02
    }

In [18]:
import pandas as pd
data = pd.read_csv(meta_path)

In [19]:
# # Fixing of files which scipy.io.wavfile chokes on.
# # This is done using ffmpeg from command line

# # This has I hope been coded in such a way that:
# # 1) The original filenames can be used, as given by the csv file
# # 2) The original data is preserved (as '*_original.wav' files)
# # 3) Running this multiple times will not cause unexpected effects

# import subprocess as sp
# import shutil

# for idx in data.index:

#     folder = 'fold' + str(data['fold'][idx]) + '/'
#     loadpath = wav_path + folder + data['slice_file_name'][idx]
    
#     try:
#         sample_rate, wav = scipy.io.wavfile.read(loadpath)
#     except:
        
#         # todo - this should really not be in this except bit...
#         old_file = loadpath
#         new_file = loadpath.replace('.wav', '_original.wav')

#         shutil.move(old_file, new_file)
#         command = ['ffmpeg', '-i', new_file, old_file]
#         sp.call(command)

#     if idx % 100 == 0:
#         print idx,

In [20]:
# load in all the data
all_spec = []
all_fold_idxs = []

for idx in data.index:
    # load the audio and convert to spec
    folder = 'fold' + str(data['fold'][idx]) + '/'
    loadpath = wav_path + folder + data['slice_file_name'][idx]
    sample_rate, wav = scipy.io.wavfile.read(loadpath)

    # just take one channel of stereo files
    if len(wav.shape) == 2:  wav = wav[:, 0]
    
    spec = frequency.spectrogram(wav.ravel(), sample_rate, **spectrogram_parameters)[0]
    spec -= np.median(spec, axis=1)[:, None]
    spec[spec<0] = 0
    spec = spec.astype(np.float32)
    spec[np.isnan(spec)] = 0
    spec[np.isneginf(spec)] = -10
    spec[np.isposinf(spec)] = 10
    
    # crop height
    ss = spec.shape[0]
    new_height = (2.0/3.0) * float(ss)
    spec = spec[:new_height, :]
    
    # resize
    zoom(spec, )
         
    # add to the list
    all_spec.append(spec.astype(np.float32))
    all_fold_idxs.append(data['fold'][idx])
    
    if idx % 100 == 0:
        print idx,


0 100 200 300 400 500 600 700 800 900 1000 1100 1200 1300 1400 1500 1600 1700 1800 1900 2000 2100 2200 2300 2400 2500 2600 2700 2800 2900 3000 3100 3200 3300 3400 3500 3600 3700 3800 3900 4000 4100 4200 4300 4400 4500 4600 4700 4800 4900 5000 5100 5200 5300 5400 5500 5600 5700 5800 5900 6000 6100 6200 6300 6400 6500 6600 6700 6800 6900 7000 7100 7200 7300 7400 7500 7600 7700 7800 7900 8000 8100 8200 8300 8400 8500 8600 8700


In [21]:
np.isinf(-np.inf)
np.isposinf

<function numpy.lib.ufunclike.isposinf>

In [22]:
# pad specs which are too short. 
# Modifying list in-place, which is not pretty but it works...
for idx in range(len(all_spec)):
    
    spec = all_spec[idx]
    
    # padding the spectrogram to be the same length...
    desired_length = int(4.0 / spectrogram_parameters['overlap'] - 1)
    
    if spec.shape[1] < desired_length:
        # wrap the spectrogram
        num_tiles = np.ceil(float(desired_length) / spec.shape[1])
        tiled = np.tile(spec, (1, num_tiles))
        new_spec = tiled[:, :desired_length]
        all_spec[idx] = new_spec
    
    elif spec.shape[1] > desired_length:
        all_spec[idx] = spec[:, :desired_length]

In [23]:
# combining all data into one array
all_spec_np = np.dstack(all_spec).transpose((2, 0, 1))
print all_spec_np.shape
S = all_spec_np.shape
all_spec_np = all_spec_np.astype(np.float32).reshape(S[0], 1, S[1], S[2])
print all_spec_np.shape

(8732, 129, 199)
(8732, 1, 129, 199)


In [24]:
Y = np.array([data['classID'][idx] for idx in data.index])
print Y.shape
print Y

(8732,)
[3 2 2 ..., 1 1 1]


In [25]:
del spec

In [27]:
def ismember(a, b):
    bind = {}
    for i, elt in enumerate(b):
        if elt not in bind:
            bind[elt] = i
    return np.array([1 if itm in bind else 0 for itm in a ]).astype(bool)


all_folds = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

for test_fold in all_folds:
    
    val_fold = (test_fold) % 10 + 1
    
    train_folds = np.setdiff1d(all_folds, [test_fold, val_fold])
    
    print "Test fold: ", test_fold
    print "Validation folds: ", val_fold
    print "Train folds: ", train_folds
    print ""
    
    # sanity checks
    assert not np.intersect1d(train_folds, test_fold)
    assert not np.intersect1d(train_folds, val_fold)
    assert val_fold != test_fold

    # extract indices corresponding to each split
    train_idxs = ismember(all_fold_idxs, train_folds)
    test_idxs = all_fold_idxs == test_fold
    val_idxs = all_fold_idxs == val_fold

    # sanity check
    assert np.any(np.logical_and.reduce((
        train_idxs, test_idxs, val_idxs))) == 0

    # for each split, combine data...
    split = {}
    print all_spec_np.shape
    print train_idxs.shape
    split['train_X'] = all_spec_np[train_idxs]
    split['test_X'] = all_spec_np[test_idxs]
    split['val_X'] = all_spec_np[val_idxs]
    split['train_y'] = Y[train_idxs]
    split['test_y'] = Y[test_idxs]
    split['val_y'] = Y[val_idxs]
    
    split_savepath = base_path + 'splits_stowell_norm/split' + str(test_fold) + '.mat'
    scipy.io.savemat(split_savepath, split)
    

Test fold:  1
Validation folds:  2
Train folds:  [ 3  4  5  6  7  8  9 10]

(8732, 1, 129, 199)
(8732,)
Test fold:  2
Validation folds:  3
Train folds:  [ 1  4  5  6  7  8  9 10]

(8732, 1, 129, 199)
(8732,)
Test fold:  3
Validation folds:  4
Train folds:  [ 1  2  5  6  7  8  9 10]

(8732, 1, 129, 199)
(8732,)
Test fold:  4
Validation folds:  5
Train folds:  [ 1  2  3  6  7  8  9 10]

(8732, 1, 129, 199)
(8732,)
Test fold:  5
Validation folds:  6
Train folds:  [ 1  2  3  4  7  8  9 10]

(8732, 1, 129, 199)
(8732,)
Test fold:  6
Validation folds:  7
Train folds:  [ 1  2  3  4  5  8  9 10]

(8732, 1, 129, 199)
(8732,)
Test fold:  7
Validation folds:  8
Train folds:  [ 1  2  3  4  5  6  9 10]

(8732, 1, 129, 199)
(8732,)
Test fold:  8
Validation folds:  9
Train folds:  [ 1  2  3  4  5  6  7 10]

(8732, 1, 129, 199)
(8732,)
Test fold:  9
Validation folds:  10
Train folds:  [1 2 3 4 5 6 7 8]

(8732, 1, 129, 199)
(8732,)
Test fold:  10
Validation folds:  1
Train folds:  [2 3 4 5 6 7 8 9]

(8

In [ ]:
for key, val in split.iteritems():
    print val.shape

In [ ]:
# now let's do lasagne!
# Or maybe save the train/test folds to disk?
